Run this file to Generate RotatingMNIST dataset under ./dataset/RotatingMNIST/ directory.

In [ ]:
import os, sys
import torch
from tqdm.auto import trange

sys.path.append('..')
from loader import get_dataloader
from torchvision import transforms

In [ ]:
data_cfg = {
    'dataset': 'MNIST',
    'root': '../dataset',
    'batch_size': 100,
    'n_workers': 4,
    'split': 'training',
    'shuffle': True,
    'digits': [3],
}

dl = get_dataloader(data_cfg)

In [ ]:
data = dl.dataset.data
targets = dl.dataset.targets

In [ ]:
EPISODES_PER_IMAGE = 1
TIME_HORIZON = 36
ANGLE_STEP = 10

data = data.repeat_interleave(EPISODES_PER_IMAGE, 0).unsqueeze(-1).repeat_interleave(TIME_HORIZON, -1)
targets = targets.repeat_interleave(EPISODES_PER_IMAGE, 0)

In [ ]:
for i_idx in trange(data.shape[0]):
    theta_init = torch.rand(()) * 360
    for j_idx in range(data.shape[-1]):
        theta = theta_init.item() + ANGLE_STEP * j_idx
        data[i_idx, :, :, :, j_idx] = transforms.functional.affine(img=data[i_idx, :, :, :, j_idx], angle=theta, translate=[0, 0], scale=1., shear=0)

In [ ]:
SAVE_PATH = '../dataset/RotatingMNIST/'
os.makedirs(SAVE_PATH, exist_ok=True)
for i in data_cfg["digits"]:
    torch.save({
        'data': data[targets == i],
        'targets': targets[targets == i]
    }, os.path.join(SAVE_PATH, f'RotatingMNIST-digit={i}.pkl'))